In [0]:
import json
import requests
import urllib

# 0. Preliminary step to get sample data

This preliminary step is reproducing Lorella's workflow Python file:
https://i-lab.public.data.uu.nl/vault-ocex/ChroniclItaly%20-%20Italian%20American%20newspapers%20corpus%20from%201898%20to%201920%5B1529330521%5D/original/
I just added a folder "data_1" to keep all files in one folder

In [0]:
mkdir 'data1'

In [0]:
# Base URL
chronam = 'https://chroniclingamerica.loc.gov/'

# Chronicling America search results
results = 'https://chroniclingamerica.loc.gov/search/pages/results/?date1=1880&date2=1920&searchType=advanced&language=ita&sequence=1&lccn=2012271201&lccn=sn85066408&lccn=sn85055164&lccn=sn85054967&lccn=sn88064299&lccn=sn84037024&lccn=sn84037025&lccn=sn86092310&proxdistance=5&state=California&state=District+of+Columbia&state=Massachusetts&state=Pennsylvania&state=Piedmont&state=Vermont&state=West+Virginia&rows=100&ortext=&proxtext=&phrasetext=&andtext=&dateFilterType=yearRange&page=11&sort=date'

# Count to keep track of downloaded files
count = 0

# Gets search results in JSON format
results_json = results + '&format=json'


In [0]:
# Returns JSON 
def get_json(url):
    data = requests.get(url)
    return(json.loads(data.content))
    
data = get_json(results_json)

In [0]:
# Cycle through JSON results
for page in data['items']:
    # Create URL
    hit = str(page['id'])
    seed = hit + 'ocr.txt'
    download_url = chronam + seed
 
    # Create file name
    file_name = download_url.replace('/', '_')
    file_name = 'data1/' + file_name[41:]
    
    # Download .txt of the page
    urllib.request.urlretrieve(download_url, str(file_name))
    count += 1

# 1. Removing stop words, numbers, and punctuation marks 

## 1.1. Grouping all texts files
A dataframe is first created to keep individual files at their initial state, and the name of each file

In [0]:
import os
import pandas as pd

In [0]:
#list of the file names
files_list = os.listdir('data1')

In [0]:
#insert file names into a df
sources = pd.DataFrame(files_list, columns=['file_name'])

In [0]:
#function to read the content of the text files
def readTxtContent(fileName):
  with open('data1/' + fileName, 'r') as file:
    return ' ' + file.read().replace('\n', ' ') + ' '

In [0]:
# adding a column to the dataframe containing file content
sources['file_content'] = sources['file_name'].apply(lambda x: readTxtContent(x))

In [21]:
# for verification purposes later, count the nr of characters for each content
sources['file_len'] = sources['file_content'].apply(lambda x: len(x))
sources['file_len'].sum()

1779770

In [0]:
# variable containing all texts together
corpus = ''
for i in range(len(sources)):
  corpus += sources['file_content'][i]

In [23]:
# check length
len(corpus)

1779770

## 1.2 Removing stop words with NLTK

In [26]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
# remove punctuation and lower case
tokens = nltk.word_tokenize(corpus)
tokenized_corpus = [w.lower() for w in tokenized_corpus if w.isalnum()]


In [0]:
# show list of default Italian stopwords
print(stopwords.words('italian'))

In [0]:
# stopwords.words('italian') can be replaced by a custom list input by the user
tokenized_corpus_without_sw = [w for w in tokenized_corpus if not w in stopwords.words('italian')]

In [39]:
print(tokenized_corpus_without_sw)

['6iornale', 'italiano', 'fatto', '3lmtal1ai4', 'thc', 'first', 'italian', 'daily', 'paper', 'on', 'p', 'asipic', 'coast', 'patrizi', 'v', 'kditokn', 'ani', 'the', 'h', 'd', 'italian', 'daily', 'amkkica', '1nts', 'offick', '12', 'montgomery', 'box', 'phovr', 'ibi', 'enlered', 'ol', 'as', '2', 'class', 'mailer', 'patria', 'k', 'umanità', 'anno', 'xvii', 'cosa', 'camerlengo', 'durante', 'interregno', 'papato', 'energica', 'tempra', 'inflessibile', 'carattere', 'cardinale', 'oreglia', 'sdegnoso', 'onori', 'pompe', 'severissimo', 'comando', 'iii', 'manto', 'stufano', 'iiki', 'vati', 'cavo', 'tir', 'rkiilie', 'ii', 'papaia', 'fino', 'nomina', 'dhl', 'niioio', 'pontkficr', 'porporati', 'componenti', 'sacro', 'collegio', 'personaggio', 'importante', 'ducato', 'momento', 'ii', 'canlinale', 'ca', 'mar', 'tengo', 'luigi', 'oreglia', 'regge', '11', 'governo', 'vaticano', 'chiesa', 'cattolica', 'finché', 'nominato', 'nuovo', 'papa', 'cardinale', 'camarlengo', 'dunque', 'onte', 'momento', 'suprema'